In [1]:
import hydra
import torch
from tqdm.notebook import tqdm
from lib.utils.config import load_config
from lib.optimizer.framework import NeuralOptimizer, NeuralOptimizer2
from lib.data.loader import load_intrinsics
from lib.data.loader import load_intrinsics
from lib.rasterizer import Rasterizer
from lib.renderer.renderer import Renderer
from lib.renderer.camera import Camera
from lib.tracker.timer import TimeTracker
from lib.utils.progress import reset_progress, close_progress

def path_to_abblation(path):
    return "_".join(path.split("/")[-3].split("_")[1:])

def eval_iterations(optimizer, datamodule, N: int = 1, value: str = "loss_param", mode="iters"):
    optimizer.max_iters = 1
    optimizer.max_optims = 1

    outer_progress = tqdm(total=N+1, desc="Iter Loop", position=0)
    total_evals = len(datamodule.val_dataset)
    inner_progress = tqdm(total=total_evals, desc="Eval Loop", leave=True, position=1)

    iters_loss = {}
    iters_time = {}

    # initial evaluation no optimization
    reset_progress(inner_progress, total_evals)
    loss = []
    for batch in datamodule.val_dataloader():
        with torch.no_grad():
            batch = optimizer.transfer_batch_to_device(batch, "cuda", 0)
            out = optimizer(batch)
            out["params"] = batch["init_params"]
            loss_info = optimizer.compute_loss(batch=batch, out=out)
            loss.append(loss_info[value])
        inner_progress.update(1)
    iters_loss[0] = torch.stack(loss)
    iters_time[0] = torch.zeros_like(iters_loss[0])
    outer_progress.update(1)
        
    # evaluation after some optimization
    for iters in range(1, N+1):
        reset_progress(inner_progress, total_evals)
        if mode == "iters":
            optimizer.max_iters = iters
        else:
            optimizer.max_optims = iters
        time_tracker = TimeTracker()
        loss = []
        for batch in datamodule.val_dataloader():
            with torch.no_grad():
                batch = optimizer.transfer_batch_to_device(batch, "cuda", 0)
                time_tracker.start("optimize")
                out = optimizer(batch)
                time_tracker.stop("optimize")
                loss_info = optimizer.compute_loss(batch=batch, out=out)
                loss.append(loss_info[value])
            inner_progress.update(1)
        loss = torch.stack(loss)
        iters_loss[iters] = loss
        iters_time[iters] = torch.stack([torch.tensor(t.time_ms) for t in list(time_tracker.tracks.values())[0]])
        outer_progress.update(1)
    close_progress([outer_progress, inner_progress])
    return iters_loss, iters_time

def load_flame_renderer():
    # instanciate similar to training
    cfg = load_config("train", ["data=synthetic"])
    K = load_intrinsics(data_dir=cfg.data.intrinsics_dir, return_tensor="pt")
    camera = Camera(
        K=K,
        width=cfg.data.width,
        height=cfg.data.height,
        near=cfg.data.near,
        far=cfg.data.far,
        scale=cfg.data.scale,
    )
    rasterizer = Rasterizer(width=camera.width, height=camera.height)
    renderer = Renderer(rasterizer=rasterizer, camera=camera)
    flame = hydra.utils.instantiate(cfg.model)
    return flame, renderer


def load_neural_optimizer(flame, renderer, path):
    cfg = load_config("train", ["data=synthetic"])
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    regularize = hydra.utils.instantiate(cfg.regularize)
    neural_optimizer = NeuralOptimizer.load_from_checkpoint(
        path,
        renderer=renderer,
        flame=flame,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        weighting=weighting,
    )
    return neural_optimizer

def load_icp_optimizer(flame, renderer, overrides, neural_optimizer2: bool = False):
    cfg = load_config("train", ["data=synthetic", "optimizer.output_dir=none"]+overrides)
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    optimizer = hydra.utils.instantiate(cfg.optimizer)
    regularize = hydra.utils.instantiate(cfg.regularize)
    if neural_optimizer2:
        cfg.framework._target_ = "lib.optimizer.framework.NeuralOptimizer2"
    icp_optimizer = hydra.utils.instantiate(
        cfg.framework,
        flame=flame,
        logger=None,
        renderer=renderer,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        optimizer=optimizer,
        weighting=weighting,
    )
    return icp_optimizer.to("cuda")

# setup the datamodule
def load_datamodule(renderer, start_frame, end_frame):
    cfg = load_config("train", ["data=synthetic"])
    datamodule = hydra.utils.instantiate(
        cfg.data,
        renderer=renderer,
        val_dataset=dict(
            start_frame=start_frame,
            end_frame=end_frame,
        ),
    )
    datamodule.setup("fit")
    return datamodule

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# settings
N = 3
value = "loss_param"  #  loss_vertices, loss_param 
start_frame = 10
end_frame = 12

# checkpoints
ours = "/home/borth/GuidedResearch/logs/2024-10-01/03-31-59_abblation_ours/checkpoints/last.ckpt"
wo_neural_prior = "/home/borth/GuidedResearch/logs/2024-10-01/00-59-15_abblation_wo_neural_prior/checkpoints/last.ckpt"
w_single_corresp = "/home/borth/GuidedResearch/logs/2024-10-01/22-07-11_abblation_w_single_corresp/checkpoints/last.ckpt"
w_single_optim = "/home/borth/GuidedResearch/logs/2024-10-01/22-07-11_abblation_w_single_optim/checkpoints/last.ckpt"
wo_neural_weights = "/home/borth/GuidedResearch/logs/2024-10-01/23-50-37_abblation_wo_neural_weights/checkpoints/last.ckpt"

# loadings
times = {}
losses = {}
flame, renderer = load_flame_renderer()
datamodule = load_datamodule(renderer, start_frame, end_frame)

Creating GL context for cuda device 0
Successfully initialized EGL version 1.5
Successfully initialized OpenGL version 4.6.0 NVIDIA 535.183.01


In [ ]:
for path in [ours, wo_neural_prior, wo_neural_weights]:
    optimizer = load_neural_optimizer(flame, renderer, path)
    loss, time = eval_iterations(optimizer, datamodule, N=N, value=value, mode="iters")
    key = path_to_abblation(path)
    times[key] = time[N].median().item()
    losses[key] = loss[N].mean().item()
    print(f"{key}: loss={losses[key]:.03f} time={times[key]:.03f}ms")

path = w_single_corresp
optimizer = load_neural_optimizer(flame, renderer, path)
loss, time = eval_iterations(optimizer, datamodule, N=N, value=value, mode="optims")
key = path_to_abblation(path)
times[key] = time[N].median().item()
losses[key] = loss[N].mean().item()
print(f"{key}: loss={losses[key]:.03f} time={times[key]:.03f}ms")

path = w_single_optim 
optimizer = load_neural_optimizer(flame, renderer, path)
loss, time = eval_iterations(optimizer, datamodule, N=N, value=value, mode="optims")
key = "abblation_wo_end_to_end"
times[key] = time[N].median().item()
losses[key] = loss[N].mean().item()
print(f"{key}: loss={losses[key]:.03f} time={times[key]:.03f}ms")
key = path_to_abblation(path)
times[key] = time[1].median().item()
losses[key] = loss[1].mean().item()
print(f"{key}: loss={losses[key]:.03f} time={times[key]:.03f}ms")

In [ ]:
optimizer = load_icp_optimizer(flame, renderer, ["residuals=face2face", "weighting=dummy", "regularize=dummy"])
loss, time = eval_iterations(optimizer, datamodule, N=N, value=value)
print("icp-baseline")
print(optimizer.time_tracker.print_summary())
print({k: v.median().item() for k, v in time.items()})
print({k: v.mean().item() for k, v in loss.items()})

In [ ]:
optimizer = load_icp_optimizer(flame, renderer, ["regularize.latent_scale=1"])
loss, time = eval_iterations(optimizer, datamodule, N=1, value=value)
print(optimizer.time_tracker.print_summary())
print({k: v.median().item() for k, v in time.items()})
print({k: v.mean().item() for k, v in loss.items()})

In [ ]:
optimizer = load_icp_optimizer(flame, renderer, ["weighting.features=64"])
loss, time = eval_iterations(optimizer, datamodule, N=1, value=value)
print(optimizer.time_tracker.print_summary())
print({k: v.median().item() for k, v in time.items()})
print({k: v.mean().item() for k, v in loss.items()})

In [6]:
optimizer = load_icp_optimizer(flame, renderer, ["regularize.latent_scale=4"], True)
loss, time = eval_iterations(optimizer, datamodule, N=1, value=value)
print(optimizer.time_tracker.print_summary())
print({k: v.median().item() for k, v in time.items()})
print({k: v.mean().item() for k, v in loss.items()})

Iter Loop:   0%|          | 0/2 [00:00<?, ?it/s]

Eval Loop:   0%|          | 0/16 [00:00<?, ?it/s]

RuntimeError: Cannot access data pointer of Tensor that doesn't have storage